# Topic Modeling with gensim
We'll try out [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) in [gensim](http://radimrehurek.com/gensim/index.html) on the [20 Newsgroups dataset](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) with some simple preprocessing.

#### Install gensim

In [1]:
# pip install --upgrade gensim

##### imports

In [1]:
from __future__ import print_function

In [2]:
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

/usr/local/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


Let's retain only a subset of the 20 categories in the original 20 Newsgroups Dataset.

In [3]:
# Set categories
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball']#, 
              #'rec.motorcycles', 'sci.space', 'talk.politics.mideast']
# Download the training subset of the 20 NG dataset, with headers, footers, quotes removed
# Only keep docs from the 6 categories above
ng_train = datasets.fetch_20newsgroups(subset='train', categories=categories, 
                                      remove=('headers', 'footers', 'quotes'))

In [4]:
# Take a look at the first doc
ng_train.data[0]

u'\n\n\nI happen to be a big fan of Jayson Stark.  He is a baseball writer for the \nPhiladelphia Inquirer.  Every tuesday he writes a "Week in Review" column.  \nHe writes about unusual situations that occured during the week.  Unusual\nstats.  He has a section called "Kinerisms of the Week" which are stupid\nlines by Mets brodcaster Ralph Kiner.  Every year he has the LGTGAH contest.\nThat stands for "Last guy to get a hit."  He also writes for Baseball \nAmerica.  That column is sort of a highlights of "Week in Review."  If you \ncan, check his column out sometime.  He might make you laugh.\n\nRob Koffler\n'

## Document Preprocessing
We'll need to generate a term-document matrix of word (token) counts for use in LDA.

We'll use `sklearn`'s `CountVectorizer` to generate our term-document matrix of counts. We'll make use of a few parameters to accomplish the following preprocessing of the text documents all within the `CountVectorizer`:
* `analyzer=word`: Tokenize by word
* `ngram_range=(1,2)`: Keep all 1 and 2-word grams
* `stop_words=english`: Remove all English stop words
* `token_pattern=\\b[a-z][a-z]+\\b`: Match all tokens with 2 or more (strictly) alphabet characters

In [5]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")
count_vectorizer.fit(ng_train.data)

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
        tokenizer=None, vocabulary=None)

In [6]:
# Create the term-document matrix
# Transpose it so the terms are the rows
counts = count_vectorizer.transform(ng_train.data).transpose()

In [7]:
counts.shape

(118115, 1661)

##### Convert to gensim
We need to convert our sparse `scipy` matrix to a `gensim`-friendly object called a Corpus:

In [8]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(counts)

##### Map matrix rows to words (tokens)
We need to save a mapping (dict) of row id to word (token) for later use by gensim:

In [9]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [10]:
len(id2word)

118115

## LDA
At this point we can simply plow ahead in creating an LDA model.  It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

In [11]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus, num_topics=3, id2word=id2word, passes=10)

Let's take a look at what happened.  Here are the 5 most important words for each of the 3 topics we found:

In [12]:
lda.print_topics()

[(0,
  u'0.002*"image" + 0.002*"jpeg" + 0.002*"graphics" + 0.002*"edu" + 0.002*"file" + 0.001*"images" + 0.001*"pub" + 0.001*"gif" + 0.001*"ftp" + 0.001*"format"'),
 (1,
  u'0.002*"people" + 0.002*"don" + 0.002*"god" + 0.002*"just" + 0.001*"think" + 0.001*"image" + 0.001*"like" + 0.001*"atheism" + 0.001*"does" + 0.001*"know"'),
 (2,
  u'0.002*"don" + 0.002*"does" + 0.002*"think" + 0.002*"know" + 0.002*"like" + 0.002*"good" + 0.001*"year" + 0.001*"just" + 0.001*"god" + 0.001*"time"')]

#### Topic Space
If we want to map our documents to the topic space we need to actually use the LdaModel transformer that we created above, like so:

In [13]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [14]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

Now we can take a look at the document vectors in the topic space, which are measures of the component of each document along each topic.  Thus, at most a document vector can have num_topics=3 nonzero components in the topic space, and most have far fewer.

In [15]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:15]

[[(0, 0.9927336384858032)],
 [(2, 0.982152467234927)],
 [(0, 0.76219679209689462), (1, 0.23112987593817744)],
 [(0, 0.021284637174004715),
  (1, 0.1418771963330436),
  (2, 0.83683816649295162)],
 [(2, 0.98749660660566974)],
 [(1, 0.99245586302368782)],
 [(0, 0.99337251384713232)],
 [(0, 0.012659039402200072),
  (1, 0.97581570966376718),
  (2, 0.011525250934032824)],
 [(0, 0.017283551295813198),
  (1, 0.96479380172954432),
  (2, 0.017922646974642506)],
 [(1, 0.98948203339383223)],
 [(0, 0.98275808620714067)],
 [(0, 0.98397097980191628)],
 [(0, 0.024810731111041293),
  (1, 0.025285634903234773),
  (2, 0.94990363398572397)],
 [(1, 0.99283808119602535)],
 [(0, 0.99292617530340865)]]

In [16]:
ng_train.data[5]

u': In article <11838@vice.ICO.TEK.COM>, bobbe@vice.ICO.TEK.COM (Robert\n: >   Someone spank me if I\'m wrong, but didn\'t Lord, Liar, or Lunatic\n: >   originate with C.S. Lewis?  Who\'s this Campollo fellow anyway?\n\n: I do think so, and isn\'t there a clear connection with the "I do\n: believe, because it is absurd" notion by one of the original\n: Christians (Origen?).\n\nThere is a similar statement attributed to Anselm, "I believe so that\nI may understand". In both cases reason is somewhat less exalted than\nanyone posting here could accept, which means that neither statement\ncan be properly analysed in this venue.'

In [17]:
lda.log_perplexity

<bound method LdaModel.log_perplexity of <gensim.models.ldamodel.LdaModel object at 0x10f5c0ad0>>

## On your own...
- Go get some of the NIPS papers from [here](https://archive.ics.uci.edu/ml/datasets/Bag+of+Words).  
- Try performing LDA on this data with gensim
- Play with some of the preprocessing options and parameters for LDA, observe what happens
- See if you can use the resulting topic space to extract topic vectors and cluster some documents
- How do your results look?